# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [25]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [26]:
cur.execute("""SELECT firstName, lastName
                FROM employees
                JOIN offices
                USING (officeCode)
                WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [27]:
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS num_employees
                FROM offices AS o
                LEFT JOIN employees AS e
                USING (officeCode)
                GROUP BY officeCode
                HAVING num_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [31]:
# employees that have helped the most customers
cur.execute("""SELECT e.employeeNumber, e.lastName, e.firstName, COUNT(c.customerNumber) AS num_customers
                FROM employees AS e
                LEFT JOIN customers AS c
                ON e.employeeNumber = c.salesRepEmployeeNumber
                GROUP BY e.employeeNumber
                ORDER BY num_customers DESC""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,employeeNumber,lastName,firstName,num_customers
0,1401,Castillo,Pamela,10
1,1504,Jones,Barry,9
2,1501,Bott,Larry,8
3,1323,Vanauf,George,8
4,1370,Hernandez,Gerard,7


In [44]:
# Employees that have sold the most merch by total $ amount
cur.execute("""SELECT c.salesRepEmployeeNumber, SUM(p.amount) as totalsales
                FROM customers AS c
                LEFT JOIN payments AS p
                USING (customerNumber)
                GROUP BY c.salesRepEmployeeNumber
                ORDER BY totalsales DESC""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)


,salesRepEmployeeNumber,totalsales
0,1370,1112003.81
1,1165,989906.55
2,1401,750201.87
3,1501,686653.25
4,1504,637672.65
5,1323,584406.80
6,1337,569485.75
7,1611,509385.82
8,1612,497907.16
9,1286,488212.67


In [48]:
# customers that have ordered the greatest quanitity of items
cur.execute("""SELECT o.customerNumber, SUM(d.quantityOrdered) AS itemsPurchased
                FROM orders as o
                JOIN orderdetails as d
                USING (orderNumber)
                GROUP BY o.customerNumber
                ORDER BY itemsPurchased DESC""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,customerNumber,itemsPurchased
0,141,9327
1,124,6366
2,114,1926
3,119,1832
4,187,1778
5,151,1775
6,323,1691
7,450,1656
8,278,1650
9,496,1647


## Level Up: Display the names of every individual product that each employee has sold

In [49]:
cur.execute("""SELECT c.salesRepEmployeeNumber, p.productName
                FROM customers AS c
                JOIN orders AS o
                USING (customerNumber)
                JOIN orderdetails AS d
                USING (orderNumber)
                JOIN products AS p
                USING (productCode)
                GROUP BY c.salesRepEmployeeNumber""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,salesRepEmployeeNumber,productName
0,1165,1958 Setra Bus
1,1166,1917 Grand Touring Sedan
2,1188,1952 Alpine Renault 1300
3,1216,2001 Ferrari Enzo
4,1286,2001 Ferrari Enzo
5,1323,1969 Harley Davidson Ultimate Chopper
6,1337,1952 Alpine Renault 1300
7,1370,1965 Aston Martin DB5
8,1401,1972 Alfa Romeo GTA
9,1501,1972 Alfa Romeo GTA


## Level Up: Display the Number of Products each employee has sold

In [51]:
cur.execute("""SELECT c.salesRepEmployeeNumber, COUNT(p.productName) AS numberProducts
                FROM customers AS c
                JOIN orders AS o
                USING (customerNumber)
                JOIN orderdetails AS d
                USING (orderNumber)
                JOIN products AS p
                USING (productCode)
                GROUP BY c.salesRepEmployeeNumber
                ORDER BY numberProducts""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,salesRepEmployeeNumber,numberProducts
0,1166,114
1,1702,114
2,1188,124
3,1621,137
4,1286,142
5,1216,152
6,1337,177
7,1611,185
8,1612,185
9,1323,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!